# Generates the JSON manifests that are used to run the SAILOR pipeline
- Use sailor/1.1.0 so that we can capture C/T edits. The current released version only does A/G

In [1]:
import yaml
import os
import pandas as pd
import glob
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook
import time

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep'

ref_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/inputs/'

In [3]:
### I'm re-running this notebook a few times on a few different groups of bam files
bams = sorted(glob.glob(os.path.join(input_dir, '*trial*.bam')))
print(len(bams))
bams[:5]

590


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-0.txt.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-1.txt.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-2.txt.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-3.txt.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/bam_file_groups/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-4.txt.bam']

# This block generates SAILOR json manifest runner files.

In [4]:
skip_rmdup = "true"
progress = tnrange(len(bams))
for bam in bams:
    genome_file = os.path.join(ref_dir, 'refdata-cellranger-hg19_lenti_common-3.0.0/fasta/genome.fa')
    snp_file = os.path.join(ref_dir, 'hg19.commonSNPs147.bed3.sc')
    
    assert os.path.exists(genome_file)
    assert os.path.exists(snp_file)
    
    sample = {"input_bam": {
        'class':'File',
        'path':bam
    }}
    fa = {"reference": {
        'class':'File',
        'path':genome_file
    }}
    known_snp = {"known_snp": {
        'class':'File',
        'path':snp_file
    }}
    json_output = os.path.join(
        output_dir, '{}.json'.format(
            os.path.splitext(os.path.basename(bam))[0]
        )
    )
    
    if not os.path.exists(json_output):
        # print(json_output)
        with open(
            json_output, 'w'
        ) as o:
            o.write("#!/usr/bin/env SAILOR-deep\n")  # use SAILOR-deep for really deep sequenced BAMs. SAILOR is OK to use for everything under ~100M reads, but SAILOR-deep increases the mpileup depth to 10M
            yaml.dump(sample, o, default_flow_style=False)
            yaml.dump(known_snp, o, default_flow_style=False)
            yaml.dump(fa, o, default_flow_style=False)
            o.write("ct: true\n")
            o.write("min_variant_coverage: 5\n")
            o.write("alpha: {}\n".format(0))
            o.write("beta: {}\n".format(0))
            o.write("edit_fraction: {}\n".format(0.01))
            o.write("skip_duplicate_removal: {}\n".format(skip_rmdup))
            o.write("reverse_stranded_library: false\n")
    progress.update(1)

# Grab all JSON files created from the above cell.
- actually just grab the ones we want to run since we're kinda doing this piece by piece. 

In [5]:
jsons = glob.glob(os.path.join(output_dir, '*.json'))

In [6]:
jsons[:3]

['/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-7_barcodes_noCS.rand10.txt.json',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand36.trial-6.txt.json',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/RBFOX2-TIA1-STAMP_round2E_read1_feature_FB_possorted_genome_bam_MD-34_barcodes.rand0.trial-3.txt.json']

In [7]:
cmds = []

progress = tnrange(len(jsons))
for json in jsons:
    if not os.path.exists(os.path.splitext(json)[0]):
        cmd = 'module load sailor/1.1.0;'
        ! chmod +x $json
        cmd += 'cd {};'.format(output_dir)
        cmd += './{};'.format(os.path.basename(json))
        cmds.append(cmd)
    progress.update(1)

cmds[:10]

[]

In [8]:
print("total commands: {}".format(len(cmds)))

total commands: 0


In [9]:
Submitter(
    commands=cmds,
    job_name='scRNA-sailor-groups_07-rand',
    sh='scRNA-sailor-groups_07-rand.sh',
    array=True,
    nodes=1,
    ppn=1,
    walltime='40:00:00',
    submit=True,
)

Writing 20 tasks as an array-job.
Wrote commands to scRNA-sailor-groups_07-rand.sh.
Submitted script to queue home.
 Job ID: 22036517
